In [1]:
import json
import pandas as pd
from confluent_kafka import Producer
from aily_py_commons.io.env_vars import (
    INFRASTRUCTURE_PROD,
    AilySettings,
)
from aily_data_aws.aws.secrets import get_secret

In [2]:
AilySettings(INFRASTRUCTURE_PROD)

secret_name = "aily/infrastructure/dev/msk/msk-controlplane-infrastructure-dev/ailypubsub"

secret = get_secret(secret_name)

# Producer configuration using AWS secrets
producer_conf = {
    'bootstrap.servers': secret["bootstrap_public"],
    'security.protocol': "SASL_SSL",
    'sasl.mechanism': "SCRAM-SHA-512",
    'sasl.username': secret["username"],
    'sasl.password': secret["password"]
}

producer = Producer(**producer_conf)

2024-06-10 14:15:04 CEST+0200 - WARNING - aily-logging: You are using a dictionary to configure AilySettings. This is not recommended.


In [3]:
def delivery_report(err, msg):
    if err is not None:
        print(f'Message delivery failed: {err}')
    else:
        print(f'Message delivered to {msg.topic()} [{msg.partition()}]')

Message delivered to langfuse-aggregator [5]


0

### Send string message

In [4]:
message = 'Hello, Kafka!'
producer.produce('kafka-message-test', message.encode('utf-8'), callback=delivery_report)

# Wait for any outstanding messages to be delivered and delivery reports to be received
producer.flush()

Message delivered to kafka-message-test [0]


0

### Send JSON mesage

We are going to send data from a simple dataframe

In [6]:
df = pd.DataFrame({
    'column1': [1, 2, 3],
    'column2': ['a', 'b', 'c']
})

# Convert DataFrame to dictionary
data_dict = df.to_dict(orient='records')  # List of dictionaries, one per row

# Serialize dictionary to JSON
message = json.dumps(data_dict)

producer.produce('kafka-message-test', message.encode('utf-8'), callback=delivery_report)

# Wait for any outstanding messages to be delivered and delivery reports to be received
producer.flush()

Message delivered to kafka-message-test [0]


0